# ID3: Algorithim Implementation from Scratch

### Main-Class
This class is responsable for holding some common attributes between the Attribute Node(internal) and the Leaf Node.


In [ ]:
class Node:
    def __init__(self, attributeValue, count):
        self.attributeValue = attributeValue
        self.count = count

### Sub-class: Attribute
This subclass extends from Node and initializes with specific attributes related to attribute handling within the tree structure.

In [ ]:
class Attribute(Node):
    def __init__(self, attribute=None, isDiscreteType=None, majorTargetValue=None, attributeValue=None, count=None):
        super().__init__(attributeValue, count)
        self.attribute = attribute 
        self.isDiscreteType = isDiscreteType
        self.majorTargetValue = majorTargetValue
        self.children = []
    
    def addLeafNode(self, attributeValue, targetValue, count):
        child = Leaf(attributeValue, targetValue, count)
        self.children.append(child)
        return child
    
    def addAttributeNode(self, attribute, isDiscreteType, majorTargetValue, attributeValue, count):
        child = Attribute(attribute, isDiscreteType, majorTargetValue, attributeValue, count)
        self.children.append(child)
        return child

### Sub-class: Leaf
This subclass inherits from Node and is designed to represent leaf nodes in the tree structure, focusing on storing target values associated with attribute values.

In [ ]:
class Leaf(Node):
    def __init__(self, targetValue, attributeValue, count=None):
        super().__init__(attributeValue, count)
        self.targetValue = targetValue

### Function `build_tree()`
This function constructs a decision tree using the ID3 algorithm, based on a given dataset (dataframe). It recursively builds the tree structure, starting from a root node (parentNode) and considering both discrete and continuous attributes.

In [ ]:
def buildTree(dataframe: pd.DataFrame, parentNode=None, uniqueValues=None):
    target, attributes = dataframe.iloc[:, -1], dataframe.iloc[:, :-1]
    bestAttribute, bestSplit = getMostInformativeFeature(attributes, target)
    isDiscreteType = True if bestSplit is None else False
    
    if attributes.empty:
        return
    elif parentNode is None: #rootNode need to be created
        parentNode = Attribute(bestAttribute, isDiscreteType, target.value_counts().idxmax(), attributeValue=None, count=len(attributes))
        uniqueValues = {col: dataframe[col].unique().tolist() for col in dataframe}
    else:
        parentNode.attribute = bestAttribute
        parentNode.isDiscreteType = isDiscreteType
    
    if isDiscreteType: #Column Type is Discrete
        currentValues = attributes[bestAttribute].unique()
        allAttributeValues = uniqueValues[bestAttribute]
        discreteChildren(dataframe, parentNode, currentValues, uniqueValues)
        leafForValueNotInCurrValues(parentNode, currentValues, allAttributeValues)
    else: #Column Type is Continuous
        continuousChildren(dataframe, parentNode, bestSplit, uniqueValues)
        
    return parentNode

### Function `continuousChildren()`

This function handles the creation of child nodes for a continuous attribute split in a decision tree, based on a given dataset (dataframe), parent node (parentNode), best split value (bestSplit), and unique attribute values (uniqueValues).

The continuousChildren function divides the dataset (dataframe) based on a continuous attribute (parentNode.attribute) into two subsets (dfLessEqual and dfGreater) using the bestSplit value. It then creates child nodes under parentNode for each subset, representing the split condition (<= bestSplit and > bestSplit).

In [ ]:
def continuousChildren(dataframe, parentNode, bestSplit, uniqueValues):
    dfLessEqual = (dataframe.loc[dataframe[parentNode.attribute] <= bestSplit]).drop(columns=parentNode.attribute)
    dfGreater = (dataframe.loc[dataframe[parentNode.attribute] > bestSplit]).drop(columns=parentNode.attribute)
    for childDataFrame in [dfLessEqual, dfGreater]: 
        if childDataFrame.equals(dfLessEqual):
            attributeValue = "<=" + str(bestSplit)
        else:
            attributeValue = ">" + str(bestSplit)
        createNewNode(parentNode, childDataFrame, attributeValue, uniqueValues)
    return

### Function `discreteChildren()` and `leafForValueNotInCurrValues()` 
This function creates child nodes under a parent node (parentNode) for each unique discrete attribute value (currentValues) in a dataset (dataframe), using the ID3 algorithm.

The discreteChildren function iterates through each unique discrete attribute value (attributeValue) in currentValues for parentNode.attribute within dataframe. It creates a child node for each subset of dataframe corresponding to attributeValue, using createNewNode to construct nodes based on uniqueValues.

Then, the second function adds leaf nodes to a parent node (parentNode) for attribute values that are not present in the current discrete attribute values (currentValues), but are part of the original dataset, ensuring completeness in the decision tree.

In [ ]:
def discreteChildren(dataframe, parentNode, currentValues, uniqueValues):
    for attributeValue in currentValues:
            childDataFrame = (dataframe[dataframe[parentNode.attribute] == attributeValue]).drop(columns=parentNode.attribute)
            
            createNewNode(parentNode, childDataFrame, attributeValue, uniqueValues)
    return

def leafForValueNotInCurrValues(parentNode, currentValues, allAttributeValues):
    for attributeValue in allAttributeValues:
        if attributeValue not in currentValues:
            parentNode.addLeafNode(attributeValue, parentNode.majorTargetValue, 0)
    return

In [ ]:
def createNewNode(parentNode, childDataFrame, attributeValue, uniqueValues):
    counts = childDataFrame.iloc[:, -1].value_counts()
    count = len(childDataFrame)
    if isNodePure(counts) or not haveAttributesAvailable(childDataFrame):
        targetValue = counts.idxmax()
        parentNode.addLeafNode(targetValue, attributeValue, count)
    else:
        attributeNode = parentNode.addAttributeNode(attribute=None, isDiscreteType=None, count=count, majorTargetValue=counts.idxmax(), attributeValue=attributeValue)   
        buildTree(childDataFrame, attributeNode, uniqueValues)
    return

def haveAttributesAvailable(childDataFrame):
    return childDataFrame.shape[1] > 1
 
def isNodePure(counts):
    return len(counts) == 1

### Function `entropy()`

The entropy function calculates the entropy of a target variable, which is a measure of disorder in the data. It is used extensively in decision tree algorithms for assessing the purity of subsets of data. This universal function serves as the core entropy calculation used across different algorithms(continuous and discrete). It ensures consistency and accuracy in entropy computations throughout the decision tree construction process.

In [ ]:
def entropy(target: pd.DataFrame):
    counts = target.value_counts()
    probs = counts/len(target)
    return np.sum(-probs * np.log2(probs))

### Function `continousEntropy()`

The continousEntropy function calculates entropy for continuous or numerical features by evaluating potential split points. For Split Entropy Calculation we divide the feature into subsets based on split points and calculates the weighted entropy of each subset using the universal entropy function.

In [ ]:
def continousEntropy(feature, target):
    dfTemp = pd.concat([feature, target], axis=1)
    entropyResults = {}
    inverseEntropyResults = {}
    uniqueValuesList = sorted(feature.unique().tolist())
    
    for value in uniqueValuesList:
        totalWeightedEntropy = splitEntropy(dfTemp, value)
        
        entropyResults[value] = totalWeightedEntropy
        inverseEntropyResults[totalWeightedEntropy] = value
    
    smallestEntropy = min(entropyResults.values())
    splitPoint = inverseEntropyResults[smallestEntropy]
    return smallestEntropy, splitPoint
            
def splitEntropy(df: pd.DataFrame, splitPoint):
    feature, target = df.columns
    dfLessEqual = df.loc[df[feature] <= splitPoint]
    dfGreater = df.loc[df[feature] > splitPoint]
    
    lessEqualEntropy = entropy(dfLessEqual[target])
    greaterEntropy = entropy(dfGreater[target])
    n = len(df)
    totalWeightedEntropy = lessEqualEntropy * len(dfLessEqual)/n + greaterEntropy * len(dfGreater)/n
    return totalWeightedEntropy


### Function `discreteEntropy()`

The discreteEntropy function calculates entropy for discrete or categorical features by evaluating each unique value recursively. It computes the weighted average entropy somatory across subsets formed by each unique value of the feature.

In [2]:
def  discreteEntropy(feature, target):
    dfTemp = pd.concat([feature, target], axis=1)
    uniqueValues = feature.unique()
    totalWeightedEntropy = 0
    
    for value in uniqueValues:
        subSet = dfTemp[dfTemp[feature.name] == value]
        
        totalWeightedEntropy += len(subSet) / len(dfTemp) * entropy(subSet[target.name])
    
    return totalWeightedEntropy

### Function `informationGain()`

The informationGain function calculates the information gain when splitting a dataset based on a specific feature. It also evaluates whether a feature is numeric or categorical. If numeric, it computes the weighted entropy after splitting using continuousEntropy, also returning the optimal split point. For categorical features, it computes the weighted entropy using discreteEntropy.

In [ ]:
def informationGain(feature, target, entrophyBefore):
    continuousSplitPoint = None
    if pd.api.types.is_numeric_dtype(feature):
        weightedEntrophyAfter, continuousSplitPoint= continousEntropy(feature, target)
    else: 
        weightedEntrophyAfter = discreteEntropy(feature, target)
    infoGain = entrophyBefore - weightedEntrophyAfter
    return infoGain, continuousSplitPoint

### Function `getMostInformativeFeature()`

The getMostInformativeFeature function identifies the most informative feature for splitting a dataset based on the highest information gain. This function iterates through each attribute in the dataset (attributes) to compute the information gain using informationGain. It compares each attribute's information gain to determine the attribute that provides the highest gain (maxInfoGain). The function returns the best attribute (bestInfoAttribute) and its associated split point (bestSplit).

In [ ]:
def getMostInformativeFeature(attributes, target):
    entrophyBefore = entropy(target)
    maxInfoGain = -1
    bestInfoAttribute = None
    
    for _, attributeCol in enumerate(attributes):
        colInfoGain, splitPoint = informationGain(attributes[attributeCol], target, entrophyBefore)
        if colInfoGain > maxInfoGain:
            maxInfoGain = colInfoGain
            bestInfoAttribute = attributeCol
            bestSplit = splitPoint
    return bestInfoAttribute, bestSplit

### Function `predictClass()` and `addPredictedColumn`

This function recursively traverses the decision tree starting from the root node (node). It checks if the current node is a leaf node (Leaf instance); if so, it returns the target value of the leaf. Otherwise, it iterates through the children of the node to find the appropriate child node based on the value of the attribute in the row (rowData). Depending on whether the attribute is discrete or continuous, it compares or evaluates the attribute values to determine the next node to traverse until a leaf node is reached.

In [ ]:
def predictClass(node, rowData):
    if isinstance(node, Leaf):
        return node.targetValue
    
    for child in node.children:
        if node.isDiscreteType:
            if child.attributeValue == rowData[node.attribute]:
                return predictClass(child, rowData)
        else:
            if child.attributeValue.startswith("<="):
                if rowData[node.attribute] <= float(child.attributeValue.lstrip("<=")):
                    return predictClass(child, rowData)
            elif child.attributeValue.startswith(">"):
                if rowData[node.attribute] > float(child.attributeValue.lstrip(">")):
                    return predictClass(child, rowData)


def addPredictedColumn(rootNode, testData: pd.DataFrame):
    predicted = []
    
    for _, row in testData.iterrows():
        predictedClass = predictClass(rootNode, row)
        predicted.append(predictedClass)
    testData['Predicted Class'] = predicted
    return testData     

def calculateAccuracy(testData):
    return accuracy_score(testData.iloc[:, -2], testData.iloc[:, -1]) 

### Function `runID3_MODE()`

The functions runID3_TRAINTEST and runID3_ALLDATA both implement the ID3 algorithm but serve different purposes. runID3_TRAINTEST evaluates model performance by splitting data into training and testing sets, building a decision tree on the training data, predicting test set labels, and assessing accuracy. . In contrast, runID3_ALLDATA trains the model on the entire dataset and focuses solely on visualizing the decision tree structure, providing a comprehensive view of the decision-making process without validation.

In [ ]:
def runID3_TRAINTEST(datasource):
    df = preProcess(pd.read_csv(datasource, keep_default_na=False)) #ESSA PARTE AQUI SOLUCIONA O PROBLEMA NO NaN, ELE DEIXA TUDO COMO VEM
    trainData, testData = train_test_split(df, test_size=0.2, random_state=42)
    rootNode = buildTree(trainData)
    printTree(rootNode, attributeChild=False)
    testData = addPredictedColumn(rootNode, testData)
    print(testData)
    buildTreeImg(rootNode, "iris")
    return calculateAccuracy(testData)

def runID3_ALLDATA(datasource):
    df = preProcess(pd.read_csv(datasource, keep_default_na=False))
    rootNode = buildTree(df)
    printTree(rootNode, attributeChild=False)
    
    buildTreeImg(rootNode, "iris")

### Function `buildTreeImg()`

The function buildTreeImg and its helper function exportTree are designed to visually represent a decision tree using the Graphviz library, primarily for graphical purposes. buildTreeImg initializes a Digraph object from Graphviz and calls exportTree with the root node of the decision tree (rootNode). It then exports the resulting graph to a PNG file named according to the dataset name (dataset_name). The function also opens a viewer to display the tree graphically.

In [4]:
def buildTreeImg(rootNode, dataset_name):
    dot = exportTree(rootNode)
    dot.render("f{dataset_name}_ID3_DT", format="png")
    dot.view()

def exportTree(node, dot=None, parent_name=None, edge_label=""):
    if dot is None:
        dot = Digraph()
    
    if isinstance(node, Attribute):
        node_label = f"{node.attribute}"
    elif isinstance(node, Leaf):
        node_label = f"{node.targetValue}\n{node.count}"
    
    curr_node_name = f"{id(node)}" 
    dot.node(curr_node_name, label=node_label, shape="ellipse" if isinstance(node, Leaf) else "box")
    
    if parent_name is not None:
        dot.edge(parent_name, curr_node_name, label=edge_label)
    
    for child in node.children:
        edge_label = child.attributeValue
        exportTree(child, dot, curr_node_name, edge_label)
                   
    return dot

### Function `preProcess()`

The preProcess function preprocesses a Pandas DataFrame (dataframe) by setting the index to the 'ID' column if it exists. This operation helps organize the data for easier access and manipulation, especially in scenarios where each row can be uniquely identified by an ID.

falar mais e pensar o que mais fazer com o preProcess

In [ ]:
def preProcess(dataframe: pd.DataFrame):
    if 'ID' in dataframe.columns:
        dataframe.set_index('ID', inplace=True)   
    return dataframe